In [ ]:
!pip install torch==0.4.1

     |████████████████████████████████| 519.5MB 34kB/s 
ERROR: torchvision 0.6.1+cu101 has requirement torch==1.5.1, but you'll have torch 0.4.1 which is incompatible.
ERROR: fastai 1.0.61 has requirement torch>=1.0.0, but you'll have torch 0.4.1 which is incompatible.
  Found existing installation: torch 1.5.1+cu101
    Uninstalling torch-1.5.1+cu101:
      Successfully uninstalled torch-1.5.1+cu101


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time
torch.__version__

'0.4.1'

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
!python -m spacy download de 

     |████████████████████████████████| 14.9MB 544kB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp36-none-any.whl size=14907056 sha256=99c9929884bf862b635c30600dfc768d52974b31c50340781a1c12172e5d8cbb
  Stored in directory: /tmp/pip-ephem-wheel-cache-zokpo6gb/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [ ]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [ ]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    """
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [ ]:
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [ ]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG))

In [ ]:
vars(train_data.examples[0])

{'src': ['.',
  'büsche',
  'vieler',
  'nähe',
  'der',
  'in',
  'freien',
  'im',
  'sind',
  'männer',
  'weiße',
  'junge',
  'zwei'],
 'trg': ['two',
  'young',
  ',',
  'white',
  'males',
  'are',
  'outside',
  'near',
  'many',
  'bushes',
  '.']}

In [ ]:
print('Length of training data is: ',len(train_data))
print('Lenght of validation data is: ',len(valid_data))
print('Lenght of test data is: ',len(test_data))

Length of training data is:  29000
Lenght of validation data is:  1014
Lenght of test data is:  1000


In [ ]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [ ]:
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (de) vocabulary: 7855
Unique tokens in target (en) vocabulary: 5893


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

In [ ]:
for t in train_iterator:
  print(t.src.shape)
  print(t.trg.shape)
  print(t.trg)
  break

torch.Size([23, 128])
torch.Size([21, 128])
tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [  14,   19,    4,  ...,    4,    4,    9],
        [  13, 1693,  192,  ...,   38,    9,   22],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]], device='cuda:0')


### Encoder 

In [ ]:
class Encoder(nn.Module):
  def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
    super().__init__()
    self.hid_dim = hid_dim
    self.n_layers = n_layers
    self.embedding = nn.Embedding(input_dim, emb_dim)
    self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
    self.dropout = nn.Dropout(dropout)
  
  def forward(self,src):
    #src = [ src len, batch size]
    embedded = self.dropout(self.embedding(src)) # embedded = [src len, batch size, emb dim]

    outputs, (hidden,cell) = self.rnn(embedded)

    #outputs = [src len, batch size, hid dim * n directions]
    # hidden = [n layers * n directions, batch size, hid dim]
    # cell = [n layers * n directions, batch size, hid dim]

    # outputs are always from the top hidden layer
    return hidden, cell

## Decoder

In [ ]:
class Decoder(nn.Module):
  def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
    super().__init__()
    self.output_dim = output_dim
    self.embedding = nn.Embedding(output_dim,emb_dim)
    self.hid_dim = hid_dim
    self.n_layers = n_layers
    self.dropout = nn.Dropout(dropout)
    self.rnn = nn.LSTM(emb_dim,hid_dim,n_layers,dropout)
    self.fc_out = nn.Linear(hid_dim,output_dim)
  
  def forward(self,input, hidden, cell):
    #input = [batch_size]
    #hidden = [n_layers * n directions, batch_size, hid_dim]
    #cell = [n_layers * n directions, batch_size, hid_dim]

    input = input.unsqueeze(0)
    embedded = self.dropout(self.embedding(input)) #embedded = [1, batch size, emb dim]
    
    output,(hidden, cell) = self.rnn(embedded,(hidden,cell))
    #output = [seq len, batch size, hid dim * n directions]
    #hidden = [n layers * n directions, batch size, hid dim] = cell

    prediction = self.fc_out(output.squeeze(0))

    return prediction, hidden, cell

## Seq2Seq

In [ ]:
class Seq2Seq(nn.Module):
  def __init__(self,encoder, decoder, device):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.device = device

    assert encoder.hid_dim == decoder.hid_dim

    assert encoder.n_layers == decoder.n_layers
  def forward(self, src, trg, teacher_forcing_ratio = 0.5):
    # src = [src len, batch size]
    # trg = [trg len, batch size]
    batch_size = trg.shape[1]
    trg_len = trg.shape[0]
    trg_vocab_size = self.decoder.output_dim

    # tensor to store the decoder outputs

    outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device) # this tensor stores the prediction for each of the sentence in the batch

    #final hidden states of the encoder

    hidden, cell = self.encoder(src)

    #first input to the decoder is the <sos> token
    input = trg[0,:]

    for t in range(1,trg_len):
      output, hidden, cell = self.decoder(input, hidden, cell) # output is the predicted token at t-1

      outputs[t] = output 

      #decide if we wnat to use teacher forcing  or not
      teacher_force = random.random() < teacher_forcing_ratio

      top1 = output.argmax(1) # get the highest predicted token from our predictions

      input = trg[t] if teacher_force else top1

    return outputs


   


## Training a seq2seq model

In [ ]:

INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
def init_weights(m):
  for name, param in m.named_parameters():
    nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (dropout): Dropout(p=0.5)
    (rnn): LSTM(256, 512, num_layers=2, bias=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
  )
)

In [ ]:
def count_parameters(model):
  '''
  Function counts the number of trainable parameters in the model
  '''
  return sum(p.numel() for p in model.parameters() if p.requires_grad)
print('The number of trainable parameters is {}'.format(count_parameters(model)))

The number of trainable parameters is 13899013


In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
  model.train()
    
  epoch_loss = 0
  
  for i, batch in enumerate(iterator):
      
      src = batch.src
      trg = batch.trg
      
      optimizer.zero_grad()
      
      output = model(src, trg)
      
      #trg = [trg len, batch size]
      #output = [trg len, batch size, output dim]
      
      output_dim = output.shape[-1]
      
      output = output[1:].view(-1, output_dim)
      trg = trg[1:].view(-1)
      
      #trg = [(trg len - 1) * batch size]
      #output = [(trg len - 1) * batch size, output dim]
      
      loss = criterion(output, trg)
      loss.backward()
        
      torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
      
      optimizer.step()
      
      epoch_loss += loss.item()
      
  return epoch_loss / len(iterator)


In [ ]:
def evaluate(model, iterator, criterion):
    
  model.eval()
  
  epoch_loss = 0
  
  with torch.no_grad():
  
      for i, batch in enumerate(iterator):

          src = batch.src
          trg = batch.trg

          output = model(src, trg, 0) #turn off teacher forcing

          #trg = [trg len, batch size]
          #output = [trg len, batch size, output dim]

          output_dim = output.shape[-1]
          
          output = output[1:].view(-1, output_dim)
          trg = trg[1:].view(-1)

          #trg = [(trg len - 1) * batch size]
          #output = [(trg len - 1) * batch size, output dim]

          loss = criterion(output, trg)
          
          epoch_loss += loss.item()
      
  return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time / 60)
  elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
  return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
  start_time = time.time()
  
  train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
  valid_loss = evaluate(model, valid_iterator, criterion)
  
  end_time = time.time()
  
  epoch_mins, epoch_secs = epoch_time(start_time, end_time)
  
  if valid_loss < best_valid_loss:
      best_valid_loss = valid_loss
      torch.save(model.state_dict(), 'tut1-model.pt')
  
  print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
  print(f'\tTrain Loss: {train_loss:.3f}')
  print(f'\t Val. Loss: {valid_loss:.3f}')

Epoch: 01 | Time: 0m 55s
	Train Loss: 5.047
	 Val. Loss: 4.943
Epoch: 02 | Time: 0m 55s
	Train Loss: 4.492
	 Val. Loss: 4.869
Epoch: 03 | Time: 0m 55s
	Train Loss: 4.177
	 Val. Loss: 4.628
Epoch: 04 | Time: 0m 55s
	Train Loss: 3.934
	 Val. Loss: 4.419
Epoch: 05 | Time: 0m 55s
	Train Loss: 3.754
	 Val. Loss: 4.260
Epoch: 06 | Time: 0m 55s
	Train Loss: 3.574
	 Val. Loss: 4.269
Epoch: 07 | Time: 0m 55s
	Train Loss: 3.453
	 Val. Loss: 4.091
Epoch: 08 | Time: 0m 55s
	Train Loss: 3.307
	 Val. Loss: 4.063
Epoch: 09 | Time: 0m 55s
	Train Loss: 3.186
	 Val. Loss: 3.972
Epoch: 10 | Time: 0m 55s
	Train Loss: 3.071
	 Val. Loss: 3.874


In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f}')

| Test Loss: 3.856
